In [1]:
pip install tensorflow-text

In [2]:
import os
import pathlib
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_text as text


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Getting files directory path**


In [4]:
path = os.getcwd()
Root_dir = os.path.abspath(os.path.join(path, os.pardir))

In [5]:
content = os.path.join(Root_dir, 'content/drive/MyDrive/umojahack')

In [6]:

path_train = os.path.join(content, 'data/train.csv')

df = pd.read_csv(path_train)
df = df[:10]
df.tail()
# df = df.drop(columns=['policy_number', 'loss_date', 'claim_id', 'claim_number'])


,claim_id,policy_number,claim_number,sum_insured,product,agent,class_of_business,risk_type,loss_date,client_type,renewal_frequency,primary_cause,secondary_cause,branch,target
5,ID_H22FLDGRJ0JR,pol000005951,claim0002227,80000.0,prod00015,ag00037,cob00034,rt00023,2019-12-03,ct0001,rf0002,pc0002,sc00003,br00002,0.00
6,ID_UQF7BGCES9AY,pol000007754,claim0007217,90000.0,prod00015,ag00064,cob00034,rt00023,2019-06-21,ct0001,rf0001,pc0007,sc00023,br00001,1185.84
7,ID_HK7WLAIP5OXH,pol000005826,claim0000984,60000.0,prod00015,ag00226,cob00034,rt00023,2019-04-30,ct0001,rf0002,pc0007,sc00022,br00002,752.00
8,ID_SHWOLB04CA2O,pol000010764,claim0013134,7000.0,prod00005,ag00044,cob00009,rt00032,2019-04-21,ct0003,rf0002,pc0007,sc00004,br00009,1637.73
9,ID_ZE58UZSYEW5L,pol000005356,claim0000060,8000000.0,prod00002,ag00064,cob00003,rt00002,2020-11-08,ct0001,rf0003,pc0002,sc00012,br00002,70515.00


In [7]:
df.head()

,claim_id,policy_number,claim_number,sum_insured,product,agent,class_of_business,risk_type,loss_date,client_type,renewal_frequency,primary_cause,secondary_cause,branch,target
0,ID_JZ7QWRHH57RO,pol000009680,claim0011359,300000.0,prod00027,ag00068,cob00031,rt00006,2021-05-13,ct0003,rf0001,pc0007,sc00022,br00006,82150.00
1,ID_KQ5TJMVLJ4EP,pol000006869,claim0005272,16000.0,prod00005,ag00226,cob00010,rt00032,2018-05-13,ct0003,rf0001,pc0007,sc00023,br00002,3780.24
2,ID_NYHI7WJGGIAE,pol000006135,claim0004147,3000.0,prod00029,ag00037,cob00010,rt00026,2018-08-18,ct0003,rf0001,pc0007,sc00024,br00002,2196.50
3,ID_ROMMDCXYUXN5,pol000008991,claim0009962,6000.0,prod00005,ag00037,cob00009,rt00032,2018-12-20,ct0003,rf0001,pc0007,sc00021,br00001,105.67
4,ID_2OTD9NX8L73D,pol000007768,claim0007346,26850.0,prod00015,ag00226,cob00034,rt00023,2018-11-07,ct0001,rf0001,pc0007,sc00021,br00001,1605.64


Select important features to use

In [8]:
target = df['target'].to_numpy()
sum_insured = df['sum_insured'].to_numpy()
x_df = df[['product','agent','class_of_business','risk_type','client_type','renewal_frequency',
    'primary_cause','secondary_cause','branch'
]]
x_train = x_df.to_numpy()

In [9]:
path = os.path.join(content, "transfomer_approach/strings_tokenizer")
reload_bert = tf.saved_model.load(path)

In [10]:
X_train = []
for x in x_train:
    tokens = reload_bert.tokenize(x)
    X_train.append(tokens)

In [11]:
data_x = []
for x in X_train:
  a = []
  for y in x:
    a.append(y[0].numpy())
  
  data_x.append(a)

data_x = np.array(data_x)

In [12]:
data_train = data_x
s_insured = sum_insured / max(sum_insured)

train_data = []
for c, x in enumerate(data_train):
  a = np.insert(data_train[c], [0], s_insured[c])
  train_data.append(a)

data_train = np.asarray(train_data)

In [13]:
MAX_TOKENS = 16

In [14]:
ds = tf.data.Dataset.from_tensor_slices((data_train, target))
# ds = ds.shuffle(128).batch(64)
ds.element_spec

(TensorSpec(shape=(10,), dtype=tf.int64, name=None),
 TensorSpec(shape=(), dtype=tf.float64, name=None))

In [15]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att_inp = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.att_out = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=2)
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, target, training):
        attn_output = self.att_inp(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        inp_out = self.layernorm2(out1 + ffn_output)

        attn_output2 = self.att_out(target, inp_out)
        attn_output2 = self.dropout1(attn_output2, training=training)
        out2 = self.layernorm1(target + attn_output2)
        ffn_output2 = self.ffn(out2)
        ffn_output2 = self.dropout2(ffn_output2, training=training)
        return self.layernorm2(out2 + ffn_output2)

In [16]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [17]:
class TransformerModel(tf.keras.Model):
  def __init__(self, num_heads, ff_dim,maxlen, vocab_size, embed_dim):
    super(TransformerModel, self).__init__()
    self.transformer = TransformerBlock(embed_dim, num_heads, ff_dim)
    self.embedding = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
    self.dense = keras.layers.Dense(vocab_size)
  def call(self, data, training):
    x , y = data
    x = self.embedding(x)
    x = self.transformer(x, y, training)
    output = self.dense(x)

    return output




In [18]:
ff_dim = 128
num_heads = 4
maxlen = 5211867675
vocab_size = reload_bert.get_vocab_size().numpy()
embed_dim = 16
model = TransformerModel(num_heads, ff_dim,maxlen, vocab_size, embed_dim)

In [19]:
optimizer = keras.optimizers.SGD(learning_rate=1e-3)
# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)


In [ ]:
epochs = 10
for epoch in range(epochs):
  print('\n Start of epoch {}'.format(epoch))
  for step, (inp, tar) in enumerate(ds):
    with tf.GradientTape() as tape:
      logits = model([inp, tar], training=True)
      loss_value = loss_fn(tar, logits)
    
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    if step % 100 == 0:
      print("training loss for one batch at step %d: %.4f" % (step, float(loss_value)))
      print("Seen so far: %s samples" % ((step + 1)))



 Start of epoch 0
